## Excercise Threshold Critique

Method 1 : Ratio of Areas Method  
Method 2: Angular Translatory Method

In [1]:
import cv2
import mediapipe as mp

columns = ['Frame', 'left_arm', 'right_arm', 'left_elbow', 'right_elbow',
                        'left_waist_leg', 'right_waist_leg', 'left_knee', 'right_knee',
                        'leftup_chest_inside', 'rightup_chest_inside', 'leftlow_chest_inside',
                        'rightlow_chest_inside', 'leg_spread','arm_length_ratio','shoulder_to_hip_ratio','shoulder_hip_heel_ratio','shoulder_angle_h','shoulder_angle_v','hip_angle_h','hip_angle_v','foot_angle_h','foot_angle_v']

In [18]:
import numpy as np

class PoseProcessor:
    """Processes pose landmarks from MediaPipe."""

    def __init__(self):
        self.all_angles = [(14, 12, 24), (13, 11, 23), (16, 14, 12), (15, 13, 11), (12, 24, 26), (11, 23, 25),
                           (24, 26, 28), (23, 25, 27), (11, 12, 24), (12, 11, 23), (26, 24, 23), (25, 23, 24),
                           (26, 24, 23, 25)]

    def process(self, results, with_index=False):
        """
        Extracts and formats keypoints from MediaPipe pose landmark data.

        Args:
          results: A MediaPipe pose detection results object.

        Returns:
          A list of dictionaries, where each dictionary represents a keypoint
          with the following properties:
            - X: X-coordinate of the keypoint.
            - Y: Y-coordinate of the keypoint.
            - Z: Z-coordinate of the keypoint (if available).
            - Visibility: Visibility score of the keypoint (0: invisible, 1: visible).
            - Landmark: Index of the keypoint.
        """

        if results.pose_landmarks:
            keypoints = []
            index = 0
            for data_point in results.pose_landmarks.landmark:
                this_xyz = {
                    'X': data_point.x,
                    'Y': data_point.y,
                    'Z': data_point.z if hasattr(data_point, 'z') else None,  # Handle optional Z-coordinate
                    'Visibility': data_point.visibility,
                    'Landmark': index,
                }
                keypoints.append(this_xyz)
                index += 1

            # return self.calculate_angles(keypoints, self.all_angles, mode='degree')
            if with_index:
                return self.calculate_angles_with_ind(keypoints, self.all_angles, mode='degree')
            else:
                return self.calculate_angles(keypoints, self.all_angles, mode='degree')
            
        else:
            # Handle the case where no pose landmarks are detected (optional)
            return None  # Or return an empty list, etc.

    def calculate_angle(self, a, b, c, mode='cosine'):
        """Calculates the angle between three 2D points."""
        a = np.array(a)  # First point
        b = np.array(b)  # Mid point
        c = np.array(c)  # End point

        # Calculate vector differences
        v1 = b - a
        v2 = c - b

        # Normalize vectors (optional for improved stability)
        v1_norm = v1 / np.linalg.norm(v1)
        v2_norm = v2 / np.linalg.norm(v2)

        # Calculate cosine of the angle
        cosine_angle = np.dot(v1_norm, v2_norm)

        if mode == 'degree':
            # Convert cosine to degree
            angle = np.degrees(np.arccos(cosine_angle))
        else:
            angle = cosine_angle

        return angle

    def calculate_leg_spread(self, a, b, c, d, mode='cosine'):
        # Find the midpoint between b and c
        mid_point_bc = [(b[0] + c[0]) / 2, (b[1] + c[1]) / 2]

        # Calculate the angle between a, (b, c), and d using the midpoint as the vertex
        angle = self.calculate_angle(a, mid_point_bc, d, mode)

        return angle

    def get_landmark_xyz(self, coordinates, landmark):
        """Retrieves the X and Y coordinates of a specific landmark."""
        for coord in coordinates:
            if coord['Landmark'] == landmark:
                return [coord['X'], coord['Y']]

        return None  # Handle the case where the landmark is not found

    def calculate_angles(self, coords, joints, mode='cosine'):
        """Calculates cosine radian values or degree values for a list of joint configurations."""
        angles = []
        for joint in joints:
            if len(joint) == 3:
                landmark_first = self.get_landmark_xyz(coords, joint[0])
                landmark_mid = self.get_landmark_xyz(coords, joint[1])
                landmark_end = self.get_landmark_xyz(coords, joint[2])

                angle = self.calculate_angle(landmark_first, landmark_mid, landmark_end, mode)
            else:
                landmark_l_knee = self.get_landmark_xyz(coords, joint[0])
                landmark_l_hip = self.get_landmark_xyz(coords, joint[1])
                landmark_r_hip = self.get_landmark_xyz(coords, joint[2])
                landmark_r_knee = self.get_landmark_xyz(coords, joint[3])

                angle = self.calculate_leg_spread(landmark_l_knee, landmark_l_hip, landmark_r_hip, landmark_r_knee,
                                                  mode)
            angles.append(angle)

        return angles
    
    def calculate_angles_with_ind(self, coords, joints, mode='cosine'):
        """Calculates cosine radian values or degree values for a list of joint configurations."""
        angles = []
        for joint_index, joint in enumerate(joints):
            if len(joint) == 3:
                landmark_first = self.get_landmark_xyz(coords, joint[0])
                landmark_mid = self.get_landmark_xyz(coords, joint[1])
                landmark_end = self.get_landmark_xyz(coords, joint[2])

                angle = self.calculate_angle(landmark_first, landmark_mid, landmark_end, mode)
            else:
                landmark_l_knee = self.get_landmark_xyz(coords, joint[0])
                landmark_l_hip = self.get_landmark_xyz(coords, joint[1])
                landmark_r_hip = self.get_landmark_xyz(coords, joint[2])
                landmark_r_knee = self.get_landmark_xyz(coords, joint[3])

                angle = self.calculate_leg_spread(landmark_l_knee, landmark_l_hip, landmark_r_hip, landmark_r_knee,
                                                mode)
            named_angle = ['left_arm', 'right_arm', 'left_elbow', 'right_elbow',
       'left_waist_leg', 'right_waist_leg', 'left_knee', 'right_knee',
       'leftup_chest_inside', 'rightup_chest_inside', 'leftlow_chest_inside',
       'rightlow_chest_inside', 'leg_spread']
            
            angles.append((named_angle[joint_index], angle))

        return angles
    
pose_processor = PoseProcessor()

### Check Video

In [10]:
# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Open video file
video_file = 'dumbbell_biceps_curls.mp4'
cap = cv2.VideoCapture(video_file)

# Loop through each frame in the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame with MediaPipe Pose model
    results = pose.process(rgb_frame)
    
    # Draw the pose landmarks on the frame
    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    # Display the frame
    cv2.imshow('MediaPipe Pose', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [20]:
# HELPER FUNCTION DERIVATED FROM THE VIDEO_PROCESSOR PIPELINE

import pandas as pd

def getTopDeviatingColumns(keypoints, columns):
        df = pd.DataFrame(columns=columns)

        for index, row in enumerate(keypoints):
            if row is None:
                continue

            vals = []
            vals.append(index)
            for i in row:
                if i is None:
                    continue
                val = i[1]
                vals.append(val)

            new_row_df = pd.DataFrame([vals], columns=df.columns)
            df = pd.concat([df, new_row_df], ignore_index=True)

        df_prop = df.iloc[:, 1:]
        percentage_change = (df_prop.std() / df_prop.mean()) * 100
        sorted_columns = percentage_change.sort_values(ascending=False)

        NOS_COLUMNS_TO_CONSIDER = 1
        print(sorted_columns)        
        return sorted_columns[:NOS_COLUMNS_TO_CONSIDER].index.tolist()

def getKeypoints(mp4_file_path):
        cap = cv2.VideoCapture(mp4_file_path)
        keypoints = []

        if not cap.isOpened():
            print(f"Error: Could not open video file '{mp4_file_path}'.")
            return

        frame_count = 0

        while True:
            ret, frame = cap.read()

            if not ret:
                break

            rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = mp.solutions.pose.Pose().process(rgb_image)

            keypoints.append(pose_processor.process(results, with_index=True))

            frame_count += 1

        cap.release()
        return len(keypoints)

In [21]:
path = 'dumbbell_biceps_curls.mp4'

keypoints = getKeypoints(path)

print(keypoints)

: 